In [18]:
# importazione librerie utili
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import normalize
from boruta import BorutaPy 
from sklearn.feature_selection import RFE
import pymrmr


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, recall_score, precision_score, matthews_corrcoef, cohen_kappa_score
from sklearn import metrics
import numpy as np

label_1='Label1-Mortality outcome'
label_2='Label2-SARS-CoV-2 nucleic acids'

#load data
data= pd.read_excel('TOT_imputed_data.xlsx', sep=';',decimal='.')
print(data.shape)
#creo un array di sole features
features = np.array([f for f in data.columns if f not in [label_1,label_2]])

#sostituire ...
#creo un array con solo i valori delle features.
X = data[features].values

#ravel() mi fa diventare l'array monodimensionale.
data[label_2]=data[label_2].map({'Positive': 0, 'Negative': 1})
data[label_1]=data[label_1].map({'Cured': 1, 'Unknown': 0,'Deceased':-1})
data

(1521, 232)


,Label1-Mortality outcome,Label2-SARS-CoV-2 nucleic acids,Age,Temb NORM,Gender,Dementia,Brain surgery,Brain stem hemorrhage,Cerebral atherosclerosis,Cerebral hemorrhage,...,Male genital organs,"Endocrine, nutritional and metabolic diseases",Pancreas,Breast,Skin and subcutaneous tissue,Rheumatology/Immunology,Psychiatric disorders,Blood,Abnormal clinical and laboratory findings,Others
0,1,0,81,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,50,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,65,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,73,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,64,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516,0,0,45,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1517,0,0,66,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1518,1,0,71,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1519,1,0,66,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
data.drop('Label1-Mortality outcome',
  axis='columns', inplace=True)

In [20]:
data

,Label2-SARS-CoV-2 nucleic acids,Age,Temb NORM,Gender,Dementia,Brain surgery,Brain stem hemorrhage,Cerebral atherosclerosis,Cerebral hemorrhage,Cerebral infarction,...,Male genital organs,"Endocrine, nutritional and metabolic diseases",Pancreas,Breast,Skin and subcutaneous tissue,Rheumatology/Immunology,Psychiatric disorders,Blood,Abnormal clinical and laboratory findings,Others
0,0,81,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,50,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,65,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,73,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,64,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516,0,45,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1517,0,66,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1518,0,71,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1519,0,66,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
label=label_2
Y=data[label].values.ravel()
Y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [22]:
mrmr=[]
union=[]
intersect=[]
current_intersect=[]
current_mrmr=[]
current_boruta=[]
parzial_features = list()
nit=100
i=0
y_pred=Y-Y
rf = RandomForestClassifier(n_estimators = 101, max_depth=11, max_features=None, class_weight="balanced_subsample")
for niter in range(nit):
    skf = StratifiedKFold(n_splits=10)
    for train_index, test_index in skf.split(X, Y):
            # mi creo i punti di train e di test
            X_train, X_test = X[train_index,:], X[test_index,:]
            #creo le label del train e quelle del test
            y_train, y_test = Y[train_index], Y[test_index]
            ###### BORUTA
            boruta_feature_selector = BorutaPy(rf, n_estimators='auto', verbose=0, max_iter = 50, perc = 50)#output=feature selezionate
            boruta_feature_selector.fit(X_train, y_train)
               
                
            X_train_selected=boruta_feature_selector.transform(X_train)#mette le feature selezionate(dall'X_train) da boruta in X_train_selected
            X_test_selected=boruta_feature_selector.transform(X_test)#mette le feature selezionate(dall'X_test) da boruta in X_test_selected
            indexes = np.where(boruta_feature_selector.support_ == True) #salvo l'array degli indiici della variabili (true selezionato, false non selezionato)

                
            result_mrmr=pymrmr.mRMR(data.iloc[train_index],'MIQ',X_train_selected.shape[1])

            for x in result_mrmr:
                current_mrmr.append(x)
                mrmr.append(x)
            

            for x in np.nditer(indexes):
                current_boruta.append(features[x])
                parzial_features.append(features[x])
            
            
            for x in features:
                if x in current_mrmr or x in current_boruta:
                    union.append(x)
            
            intersection_set = set.intersection(set(current_mrmr), set(current_boruta))
            current_intersect = list(intersection_set)
            intersect=intersect+current_intersect
            
            
            
            current_mrmr.clear()
            current_boruta.clear()
            current_intersect.clear()
            intersection_set.clear()
            clf = rf.fit(X_train_selected, y_train)

            y_pred[test_index] =clf.predict(X_test_selected)
               # array = boruta_feature_selector.transform(X)
            i=i+1
            print(i)




1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [23]:
#BORUTA RESULTS
final_features = list()
final_count= list()
print()
contaboruta=[0]*len(features)
a=0
for i in features:
    c=parzial_features.count(i)
    contaboruta[a]=c  
    print(i,'        ', c, c/(nit*10))
    final_features.append(i)
    final_count.append(c)
final_BORUTA = pd.DataFrame(np.column_stack([final_features, final_count]), 
                               columns=['features', 'BORUTA'])


Age          1000 1.0
Temb NORM          1000 1.0
Gender          1000 1.0
Dementia          0 0.0
Brain surgery          0 0.0
Brain stem hemorrhage          0 0.0
Cerebral atherosclerosis          0 0.0
Cerebral hemorrhage          0 0.0
Cerebral infarction          800 0.8
Intracranial aneurysm          0 0.0
Lacunar infarction          3 0.003
Aphasia          0 0.0
Organic mental disorders          0 0.0
Pituitary adenoma          0 0.0
Subdural effusion          0 0.0
Subdural hemorrhage          0 0.0
Parkinson Disease          40 0.04
Encephalopathy          0 0.0
Myasthenia gravis          0 0.0
Gastrointestinal surgery for neoplasia          488 0.488
Adjuvant Chemotherapy for gastrointestinal tumor          0 0.0
Intestinal obstruction          0 0.0
Radio-chemotherapy          0 0.0
Gastritis          39 0.039
Non specified chronic stomach disease          0 0.0
Gastrointestinal bleeding          0 0.0
Gastrointestinal Ulcer          0 0.0
Gastroesophageal reflux          

In [24]:
#MRMR RESULTS
final_features = list()
final_count= list()
print()
contamrmr=[0]*len(features)
a=0
for i in features:
    c=mrmr.count(i)
    contamrmr[a]=c  
    print(i,'        ', c, c/(nit*10))
    final_features.append(i)
    final_count.append(c)
final_MRMR = pd.DataFrame(np.column_stack([final_features, final_count]), 
                               columns=['features', 'MRMR'])


Age          900 0.9
Temb NORM          1000 1.0
Gender          242 0.242
Dementia          100 0.1
Brain surgery          1000 1.0
Brain stem hemorrhage          800 0.8
Cerebral atherosclerosis          523 0.523
Cerebral hemorrhage          100 0.1
Cerebral infarction          1000 1.0
Intracranial aneurysm          0 0.0
Lacunar infarction          0 0.0
Aphasia          103 0.103
Organic mental disorders          0 0.0
Pituitary adenoma          0 0.0
Subdural effusion          0 0.0
Subdural hemorrhage          900 0.9
Parkinson Disease          100 0.1
Encephalopathy          0 0.0
Myasthenia gravis          486 0.486
Gastrointestinal surgery for neoplasia          0 0.0
Adjuvant Chemotherapy for gastrointestinal tumor          900 0.9
Intestinal obstruction          824 0.824
Radio-chemotherapy          400 0.4
Gastritis          115 0.115
Non specified chronic stomach disease          900 0.9
Gastrointestinal bleeding          2 0.002
Gastrointestinal Ulcer          100 0.1


In [25]:
#UNION RESULTS
final_features = list()
final_count= list()
print()
contaunion=[0]*len(features)
a=0
for i in features:
    c=union.count(i)
    contaunion[a]=c  
    print(i,'        ', c, c/(nit*10))
    final_features.append(i)
    final_count.append(c)
final_UNION = pd.DataFrame(np.column_stack([final_features, final_count]), 
                               columns=['features', 'UNION'])


Age          1000 1.0
Temb NORM          1000 1.0
Gender          1000 1.0
Dementia          100 0.1
Brain surgery          1000 1.0
Brain stem hemorrhage          800 0.8
Cerebral atherosclerosis          523 0.523
Cerebral hemorrhage          100 0.1
Cerebral infarction          1000 1.0
Intracranial aneurysm          0 0.0
Lacunar infarction          3 0.003
Aphasia          103 0.103
Organic mental disorders          0 0.0
Pituitary adenoma          0 0.0
Subdural effusion          0 0.0
Subdural hemorrhage          900 0.9
Parkinson Disease          139 0.139
Encephalopathy          0 0.0
Myasthenia gravis          486 0.486
Gastrointestinal surgery for neoplasia          488 0.488
Adjuvant Chemotherapy for gastrointestinal tumor          900 0.9
Intestinal obstruction          824 0.824
Radio-chemotherapy          400 0.4
Gastritis          150 0.15
Non specified chronic stomach disease          900 0.9
Gastrointestinal bleeding          2 0.002
Gastrointestinal Ulcer          1

In [26]:
#INTERSECT RESULTS
final_features = list()
final_count= list()
print()
containtersect=[0]*len(features)
a=0
for i in features:
    c=intersect.count(i)
    containtersect[a]=c  
    print(i,'        ', c, c/(nit*10))
    final_features.append(i)
    final_count.append(c)
final_INTERSECT = pd.DataFrame(np.column_stack([final_features, final_count]), 
                               columns=['features', 'INTERSECT'])


Age          900 0.9
Temb NORM          1000 1.0
Gender          242 0.242
Dementia          0 0.0
Brain surgery          0 0.0
Brain stem hemorrhage          0 0.0
Cerebral atherosclerosis          0 0.0
Cerebral hemorrhage          0 0.0
Cerebral infarction          800 0.8
Intracranial aneurysm          0 0.0
Lacunar infarction          0 0.0
Aphasia          0 0.0
Organic mental disorders          0 0.0
Pituitary adenoma          0 0.0
Subdural effusion          0 0.0
Subdural hemorrhage          0 0.0
Parkinson Disease          1 0.001
Encephalopathy          0 0.0
Myasthenia gravis          0 0.0
Gastrointestinal surgery for neoplasia          0 0.0
Adjuvant Chemotherapy for gastrointestinal tumor          0 0.0
Intestinal obstruction          0 0.0
Radio-chemotherapy          0 0.0
Gastritis          4 0.004
Non specified chronic stomach disease          0 0.0
Gastrointestinal bleeding          0 0.0
Gastrointestinal Ulcer          0 0.0
Gastroesophageal reflux          0 0.0
G

In [27]:
final_BORUTA.to_excel(r'L2_B.xlsx', index = False)
final_MRMR.to_excel(r'L2_M.xlsx', index = False)
final_UNION.to_excel(r'L2_U.xlsx', index = False)
final_INTERSECT.to_excel(r'L1_I.xlsx', index = False)

In [28]:
final_COMPLETE=pd.merge(final_BORUTA,final_MRMR)
final_COMPLETE=pd.merge(final_COMPLETE,final_UNION)
final_COMPLETE=pd.merge(final_COMPLETE,final_INTERSECT)
final_COMPLETE

,features,BORUTA,MRMR,UNION,INTERSECT
0,Age,1000,900,1000,900
1,Temb NORM,1000,1000,1000,1000
2,Gender,1000,242,1000,242
3,Dementia,0,100,100,0
4,Brain surgery,0,1000,1000,0
...,...,...,...,...,...
225,Rheumatology/Immunology,593,0,593,0
226,Psychiatric disorders,0,100,100,0
227,Blood,1000,189,1000,189
228,Abnormal clinical and laboratory findings,370,0,370,0


In [ ]:
parzial_features

In [ ]:
mrmr

In [ ]:
intersect

In [29]:
final_COMPLETE.to_excel(r'L2_COMP.xlsx', index = False)